In [60]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = 'browser'

In [61]:
data = sqlite3.connect("../data/checking-logs.sqlite")

In [62]:
sql_request1 = """
SELECT uid, timestamp, numTrials FROM checker
WHERE uid LIKE "user_%" AND status = "ready"
AND labname = "project1"
"""

dataframe_checker = pd.read_sql(sql_request1, data, parse_dates="timestamp")

In [63]:
dataframe_checker

,uid,timestamp,numTrials
0,user_4,2020-04-17 05:19:02.744528,1
1,user_4,2020-04-17 05:22:45.549397,2
2,user_4,2020-04-17 05:34:24.422370,3
3,user_4,2020-04-17 05:43:27.773992,4
4,user_4,2020-04-17 05:46:32.275104,5
...,...,...,...
946,user_19,2020-05-15 10:22:39.698523,26
947,user_19,2020-05-15 10:22:46.248162,27
948,user_19,2020-05-15 10:23:18.043212,28
949,user_28,2020-05-15 10:38:14.430013,27


In [64]:
data.close()

In [65]:
days = dataframe_checker.groupby(["timestamp"]).max("numTrials").reset_index()

In [66]:
days["day"] = days.index.values

dataframe_checker = pd.merge(dataframe_checker, days, on="timestamp", suffixes=("", "_y"))

In [67]:
dataframe_checker

,uid,timestamp,numTrials,numTrials_y,day
0,user_4,2020-04-17 05:19:02.744528,1,1,0
1,user_4,2020-04-17 05:22:45.549397,2,2,1
2,user_4,2020-04-17 05:34:24.422370,3,3,2
3,user_4,2020-04-17 05:43:27.773992,4,4,3
4,user_4,2020-04-17 05:46:32.275104,5,5,4
...,...,...,...,...,...
946,user_19,2020-05-15 10:22:39.698523,26,26,946
947,user_19,2020-05-15 10:22:46.248162,27,27,947
948,user_19,2020-05-15 10:23:18.043212,28,28,948
949,user_28,2020-05-15 10:38:14.430013,27,27,949


In [68]:
dataframe_checker = dataframe_checker.groupby(["uid", "day"])["numTrials"].max().rename("max").reset_index()

In [69]:
dataframe_checker

,uid,day,max
0,user_1,886,1
1,user_1,887,2
2,user_1,888,3
3,user_1,900,4
4,user_1,901,5
...,...,...,...
946,user_8,768,4
947,user_8,794,5
948,user_8,833,6
949,user_8,835,7


In [70]:
pivot = dataframe_checker.pivot(index="uid", columns="day", values="max")

pivot = pivot.ffill(axis=1)

pivot = pivot.fillna(0)

pivot = pivot.astype(int)

In [71]:
pivot

day,0,1,2,3,4,5,6,7,8,9,...,941,942,943,944,945,946,947,948,949,950
uid,,,,,,,,,,,,,,,,,,,,,
user_1,0,0,0,0,0,0,0,0,0,0,...,10,10,11,11,11,11,11,11,11,11
user_10,0,0,0,0,0,0,0,0,0,0,...,59,59,59,59,59,59,59,59,59,59
user_11,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
user_12,0,0,0,0,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4
user_13,0,0,0,0,0,0,0,0,0,0,...,32,32,32,32,32,32,32,32,32,32
user_14,0,0,0,0,0,0,0,0,0,0,...,99,99,99,99,99,99,99,99,99,99
user_15,0,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
user_16,0,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
user_17,0,0,0,0,0,0,0,1,1,1,...,6,6,6,6,6,6,6,6,6,6


In [72]:
initial_data = [
    go.Scatter(
        x=pivot.columns,
        y=pivot.loc[uid],
        name=uid,
        mode="lines + markers"
    )
    for uid in pivot.index
]

print(initial_data[:3])

[Scatter({
    'mode': 'lines+markers',
    'name': 'user_1',
    'x': array([  0,   1,   2, ..., 948, 949, 950], shape=(951,)),
    'y': array([ 0,  0,  0, ..., 11, 11, 11], shape=(951,))
}), Scatter({
    'mode': 'lines+markers',
    'name': 'user_10',
    'x': array([  0,   1,   2, ..., 948, 949, 950], shape=(951,)),
    'y': array([ 0,  0,  0, ..., 59, 59, 59], shape=(951,))
}), Scatter({
    'mode': 'lines+markers',
    'name': 'user_11',
    'x': array([  0,   1,   2, ..., 948, 949, 950], shape=(951,)),
    'y': array([0, 0, 0, ..., 1, 1, 1], shape=(951,))
})]


In [73]:
frames = []

In [74]:
for i in range(1, len(pivot) + 1):
    frame_data = [
        go.Scatter(
            x=pivot.columns[:i],
            y=pivot.loc[uid][:i],
            mode="lines + markers",
            name=uid
        )
        for uid in pivot.index
    ]

frames.append(go.Frame(data=frame_data, name=str(i)))

In [75]:
xa = len(pivot.columns)

ya = pivot.values.max() + 2

In [76]:
layout = go.Layout(
    title="Dynamic of commits per user in project1",
    xaxis=dict(range=(0, xa)),
    yaxis=dict(range=(0, ya)),
    updatemenus=[{
        "type": "buttons",
        "buttons": [{
                    "method": "animate",
                    "label": "play",
                    "args": [None]}]
    }]
)

In [77]:
figure = go.Figure(data=initial_data, layout=layout, frames=frames)

figure.show()

Gtk-Message: 14:47:24.833: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
